In [1]:
import matplotlib
import scipy 
import sys
import argparse
import numpy
import dadi
from dadi import Numerics, PhiManip, Integration, Spectrum, Misc, Demographics1D
import dadi.Godambe

In [2]:
fs= dadi.Spectrum.from_file("/Users/sergio/Downloads/Vaquita-26.sfs")
ns = fs.sample_sizes # get sample size from SFS (in haploids)
pts_l = [ns[0]+5,ns[0]+15,ns[0]+25] # this should be slightly larger (+5) than sample size and increase by 10
maxiter= 100

In [4]:
#Bootstraping
datafile = '/Users/sergio/Documents/SNPs_for_SFS_concat_vaquita.vcf.gz'
dd = dadi.Misc.make_data_dict_vcf(datafile, '/Users/sergio/Documents/Vaquita_samples.txt')

In [5]:
# Generate 100 bootstrap datasets, by dividing the genome into 2 Mb chunks and
# resampling from those chunks.
Nboot, chunk_size = 1000, 2e6
chunks = dadi.Misc.fragment_data_dict(dd, chunk_size)

In [6]:
pop_ids, ns = ['Vaquita'], [26]
boots = dadi.Misc.bootstraps_from_dd_chunks(chunks, Nboot, pop_ids, ns, polarized=False)

In [7]:
func = Demographics1D.two_epoch
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [8]:
# nu, T
popt = [0.625938354,0.240982617] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [6.73019708e-03 2.64800956e-02 7.27355851e+02]


In [9]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [4.88464633e-02 1.50486079e-01 5.68815616e+03]


In [10]:
def bottleneck(params, ns, pts):
    nuB,nuF,TB,TF = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = Integration.one_pop(phi, xx, TB, nuB)  # bottleneck
    phi = Integration.one_pop(phi, xx, TF, nuF) # recovery
    fs = Spectrum.from_phi(phi, ns, (xx,))
    return fs
func=bottleneck
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [11]:
# nuB, nuF, TB, TF
popt = [0.06796994,0.699764842,0.004888897, 0.077023972] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [           nan 2.45909490e-02            nan 9.99009682e-03
 3.26610665e+02]


In [12]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.02186778e-02 7.06140794e-02 1.02957311e-03 3.54672169e-02
 2.91297065e+03]


In [13]:
def bottleneck(params, ns, pts):
    nuB,nuF,nuC,TB,TF,TC = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = Integration.one_pop(phi, xx, TB, nuB)  # bottleneck
    phi = Integration.one_pop(phi, xx, TF, nuF) # recovery
    phi = Integration.one_pop(phi, xx, TC, nuC) # current
    fs = Spectrum.from_phi(phi, ns, (xx,))
    return fs
func=bottleneck
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [14]:
# nuB,nuR,nuC,TB,TR,TC
popt = [2.829319576,0.025202708,0.801421785,0.466126545,0.004455258,0.109345731] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [       nan 0.00239827        nan        nan        nan        nan
        nan]


In [15]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.70397957e+00 4.21591743e-03 6.86246636e-02 3.33306295e-01
 1.16019642e-03 8.42939316e-02 4.82923786e+03]
